In [171]:
import os
from PIL import Image
import pyocr
import numpy as np
import pyocr.builders
import matplotlib.pyplot as plt
import math
import difflib
import json
import time

In [172]:
path_tesseract = "C:\\Program Files\\Tesseract-OCR"
if path_tesseract not in os.environ["PATH"].split(os.pathsep):
    os.environ["PATH"] += os.pathsep + path_tesseract

tools = pyocr.get_available_tools()
tool = tools[0]

In [173]:
targets = ["normal", "wide", "fastlate"]

In [174]:
def difficult(target):
    img = Image.open("../targets/" + target + ".png")
    
    rgb_img = img.convert('RGB')
    size = rgb_img.size

    crop_img = rgb_img.crop([0, 0, size[0]/2, size[1]/7])
    crop_size = crop_img.size

    resize_img = crop_img.resize((math.floor(crop_size[0]/15), math.floor(crop_size[1]/15)))
    
    img2 = Image.new('RGBA',crop_size)

    for x in range(crop_size[0]):
        for y in range(crop_size[1]):
            r,g,b = crop_img.getpixel((x,y))

            if 94<r<175 and 179<g<255 and 28<b<108:
                a = 0
            elif 54<r<134 and 144<g<224 and 192<b<255:
                a = 0
            elif 204<r<255 and 135<g<215 and 21<b< 101:
                a = 0
            elif 180<r<255 and 42<g<122 and 64<b<144:
                a = 0
            elif 132<r<212 and 22<g<102 and 190<b<255:
                a = 0
            else:
                a = 255

            img2.putpixel((x,y),(a,a,a,255))
    
    builder = pyocr.builders.TextBuilder(tesseract_layout=6)
    builder.tesseract_configs.append("-c")
    builder.tesseract_configs.append("tessedit_char_whitelist=\"EASYNORMLHDXPT\"")
    data = tool.image_to_string(img2, lang="eng", builder=builder)
    
    return data

EASY = 134,219,68
NORMAL = 94,184,232
HARD = 244,175,61
EXPERT = 205,91,103
MASTER = 172,62,230

In [175]:
difficult("wide")

'SEXPERT'

In [179]:
def check(target):
    difficults = ["EASY","NORMAL","HARD","EXPERT","MASTER"]

    data = {"credibility":0,"difficult":""}

    for j in difficults:
        result = difflib.SequenceMatcher(None, target, j).ratio()

        if result > data["credibility"]:
            data["difficult"] = j
            data["credibility"] = result

    return data

In [180]:
start = time.perf_counter()
for target in targets:
    data = difficult(target)
    print(data)
    print(check(data))
    print(target + ": " + str(time.perf_counter() - start) + "\n")
print("avg: " + str((time.perf_counter() - start) / 3))

MASTER S
{'credibility': 0.8571428571428571, 'difficult': 'MASTER'}
normal: 0.8092503999941982

SEXPERT
{'credibility': 0.9230769230769231, 'difficult': 'EXPERT'}
wide: 1.4523073000018485

LAASTER
{'credibility': 0.7692307692307693, 'difficult': 'MASTER'}
fastlate: 1.9899605000100564

avg: 0.6633570000024823
